In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from bs4 import BeautifulSoup
import sqlite3
import json

In [5]:
con = sqlite3.connect('ziengs.db')

with con:
    
    cur = con.cursor()    
                 
    with open('crawl_ziengs.nl_2016-05-30T23-15-20.jl') as file:
        
        product_pages = []
        product_listings = []
        
        pdp_pages = 0
        p_listings = 0
        other = 0
        
        for nr, line in enumerate(file):

            data = json.loads(line)
            soup = BeautifulSoup(data['body'],"lxml")
            
            if data['page_type'] == 'product_detail':
                pdp_pages += 1
            elif data['page_type'] == 'product_listing':
                p_listings += 1
            else:
                other += 1

            if data['page_type'] == 'product_detail':           

                product_id = soup.find("input",{"id":"hdnProductId"}).get("value")
                title = soup.find("h1", {"itemprop":"name"}).contents[0]
                category = soup.find("meta",{"itemprop":"category"}).get("content")
                brand = soup.find("meta", {"itemprop":"brand"}).get("content")
                price = soup.find("meta", {"itemprop":"price"}).get("content")
                page_url = data['page_url']
                image = soup.find("meta", {"property":"og:image"}).get("content")
                page_type = data['page_type']
                crawled_at = data['crawled_at']
                try: 
                    details = soup.find("div", {"class":"usp"}).contents[0]
                    if not isinstance(details,str): 
                         details = details[0]
                except: details = ' '

                product_pages.append((product_id,title,category,brand,price,page_url,image,page_type,crawled_at,details))

            if data['page_type'] == 'product_listing':        

                nr_products = try_except(soup.find("div",{"class":"productCount"}).find("span").contents[0].strip())

                items = try_except(soup.findAll("div",{"class":["item","item last","item vanvoor","item vanvoor last"]}))
                
                crawled_at = data['crawled_at']
                product_category = ", ".join(data['product_category'])
                ordering = data['ordering']
                page_number = data['page_number']
                page_type = data['page_type']
                page_url = data['page_url']
                
                position = 0
                for item in items:
                    product_id = item.get("rel")
                    if product_id is not None:
                        position += 1
                        product_listings.append((page_url, nr_products, page_number, ordering, product_id, product_category, page_type, crawled_at))
                
            if nr%100 == 0:
                print "nr: " + str(nr)
                
                print "pdp_pages: " + str(pdp_pages)
                print "p_listings: " + str(p_listings)
                print "other: " + str(other)
                
                cur.executemany('insert into product_pages values (?,?,?,?,?,?,?,?,?,?)', product_pages)
                cur.executemany('insert into product_listings values (?,?,?,?,?,?,?,?)', product_listings)

                con.commit()
                product_pages = []
                product_listings = []
            
        cur.executemany('insert into product_pages values (?,?,?,?,?,?,?,?,?,?)', product_pages)
        cur.executemany('insert into product_listings values (?,?,?,?,?,?,?,?)', product_listings)
        con.commit()
        
        print "pdp_pages: " + str(pdp_pages)
        print "p_listings: " + str(p_listings)
        print "other: " + str(other)
        
        file.close()
    
con.close()

nr: 0
pdp_pages: 0
p_listings: 1
other: 0
nr: 100
pdp_pages: 78
p_listings: 23
other: 0
nr: 200
pdp_pages: 171
p_listings: 30
other: 0
nr: 300
pdp_pages: 266
p_listings: 35
other: 0
nr: 400
pdp_pages: 362
p_listings: 39
other: 0
nr: 500
pdp_pages: 458
p_listings: 43
other: 0
nr: 600
pdp_pages: 555
p_listings: 46
other: 0
nr: 700
pdp_pages: 649
p_listings: 52
other: 0
nr: 800
pdp_pages: 745
p_listings: 56
other: 0
nr: 900
pdp_pages: 839
p_listings: 62
other: 0
nr: 1000
pdp_pages: 935
p_listings: 66
other: 0
nr: 1100
pdp_pages: 1032
p_listings: 69
other: 0
nr: 1200
pdp_pages: 1128
p_listings: 73
other: 0
nr: 1300
pdp_pages: 1225
p_listings: 76
other: 0
nr: 1400
pdp_pages: 1318
p_listings: 83
other: 0
nr: 1500
pdp_pages: 1407
p_listings: 94
other: 0
nr: 1600
pdp_pages: 1493
p_listings: 108
other: 0
nr: 1700
pdp_pages: 1590
p_listings: 111
other: 0
nr: 1800
pdp_pages: 1682
p_listings: 119
other: 0
nr: 1900
pdp_pages: 1772
p_listings: 129
other: 0
nr: 2000
pdp_pages: 1858
p_listings: 143
ot

In [8]:
import pandas as pd

con = sqlite3.connect("ziengs.db")
df_pdp = pd.read_sql_query("SELECT * from product_pages", con)
df_pl = pd.read_sql_query("SELECT * from product_listings", con)

con.close()

In [9]:
df_pdp.head()

,product_id,product_title,product_category,brand,price,url,image_url,pagetype,crawl_date_time,description
0,10443,Rohde Pantoffel Zwart,Sandalen & slippers,Rohde,44.95,http://www.ziengs.nl/rohde-pantoffel_zwart_104...,http://www.ziengs.nl/img/rohde-pantoffel-zwart...,product_detail,2016-05-30 23:15:22,Z
1,11346,Ecco Veterbootie Cognac,Boots,Ecco,99.95,http://www.ziengs.nl/ecco-veterbootie_cognac_1...,http://www.ziengs.nl/img/ecco-veterbootie-cogn...,product_detail,2016-05-30 23:15:22,C
2,11207,Rohde Pantoffel Zwart,Sandalen & slippers,Rohde,39.95,http://www.ziengs.nl/rohde-pantoffel_zwart_112...,http://www.ziengs.nl/img/rohde-pantoffel-zwart...,product_detail,2016-05-30 23:15:22,Z
3,12261,Ecco Slipper Blauw,Sandalen & slippers,Ecco,49.99,http://www.ziengs.nl/ecco-slipper_blauw_12261....,http://www.ziengs.nl/img/ecco-slipper-blauw_20...,product_detail,2016-05-30 23:15:22,B
4,12261,Ecco Slipper Zwart,Sandalen & slippers,Ecco,49.99,http://www.ziengs.nl/ecco-slipper_zwart_12261....,http://www.ziengs.nl/img/ecco-slipper-zwart_20...,product_detail,2016-05-30 23:15:22,Z


In [25]:
df.image_url[1] #urls don't seem completely correct for the product...

u'http://www.ziengs.nl/img/gabor-instapper-rood_200x200_60277.png'

In [10]:
df_pl.head()

,page_url,nr_products,page_number,ordering,item,product_category,page_type,crawled_date_time
0,http://www.ziengs.nl/herenschoenen/pantoffels/...,3,1,popularity,10443,"Heren, Pantoffels",product_listing,2016-05-30 23:15:20
1,http://www.ziengs.nl/herenschoenen/pantoffels/...,3,1,popularity,11133,"Heren, Pantoffels",product_listing,2016-05-30 23:15:20
2,http://www.ziengs.nl/herenschoenen/pantoffels/...,3,1,popularity,11207,"Heren, Pantoffels",product_listing,2016-05-30 23:15:20
3,http://www.ziengs.nl/damesschoenen/sneakers/?t...,384,1,popularity,11341,"Dames, Sneakers",product_listing,2016-05-30 23:15:20
4,http://www.ziengs.nl/damesschoenen/sneakers/?t...,384,1,popularity,11833,"Dames, Sneakers",product_listing,2016-05-30 23:15:20


In [15]:
# product_id's zijn niet uniek *snik*
for column in df_pdp.columns:
    print column
    print df_pdp[column].value_counts(dropna=False).head(5)
    print "\n"

product_id
9823     7
12260    6
9956     5
11871    4
12182    4
Name: product_id, dtype: int64


product_title
 Ecco Sneaker Zwart         43
 Ecco Veterschoen Zwart     27
 Ecco Veterbootie Zwart     27
 Ecco Sneaker Grijs         24
 Ecco Instapper Zwart       20
Name: product_title, dtype: int64


product_category
Sneakers         364
Veterschoenen    359
Enkellaarsjes    350
Boots            219
Sandalen         193
Name: product_category, dtype: int64


brand
Ecco           453
Tamaris        371
Birkenstock    183
Cellini        113
Gabor          108
Name: brand, dtype: int64


price
69.95    351
59.95    289
79.95    282
99.95    226
49.95    226
Name: price, dtype: int64


url
http://www.ziengs.nl/ann-rocks-slipper_zwart_12068.html        1
http://www.ziengs.nl/aqa-enkellaars_cognac_10879.html          1
http://www.ziengs.nl/jack-jones-veterbootie_bruin_7372.html    1
http://www.ziengs.nl/ecco-sandaal_zwart_12064.html             1
http://www.ziengs.nl/macarena-espadrille_ta